# Mongo notebook to update fields

##  DO NOT RUN THIS STRAIGHT UP 

## IF YOU MAKE CHANGES DOCUMENENT THEM AND ENCAPSULATE THEM IN A FUNCTION THAT WONT BE RAN TO SAFEGUARD 

In [7]:
import yaml
with open("../../secrets.yaml", 'r') as f:
    config = yaml.safe_load(f)


In [8]:
from pymongo import MongoClient
client = MongoClient(config["mongoServer"])


Update analysis tables 

In [20]:
tables = ['techs','education','ops','packages']
def changeTables():
    for table in tables: 
        curTable = client.prod[table]
        query = curTable.find({},{"_id":0,"url":1})
        for item in query: 
            url = item.get("url")
            region = client.prod.Scraped_Data.find_one({"url":url},{"_id":0,"region":1})    
            curTable.update_one({"url":url},{'$set':region})

In [32]:
curTable = client.prod['model_outputs']
query = curTable.find({"country": "Canada","role":"Machine Learning"},{"_id":0,"urls":1})
k = 0

for i in query: 
    url = i.get("urls")
    for table in ['education','ops','techs','packages']:
        
        q = client.prod[table].find({"url":url})  

    
        exists = [x for x in q]
      
        
        if exists and exists[0].get("Country") != "Canada":
            
            res = client.prod[table].update_one({"url":url},{"$set":{"Country":"Canada"}})
            print(exists)
    
            
    

    

In [6]:

    curTable = client.prod['Scraped_Data']
    query = curTable.find({},{"url":1,"title":1,"country":1})
    for item in query: 
        url = item.get("url")
        for table in ['education','ops','techs','packages']:
           
            role = item.get('title')
            country = item.get('country')
            q = client.prod[table].find({"url":url})
            exists = [x for x in q]

            if exists: 
                client.prod[table].update_one({"urls":url},{'$set':{'role':role,"Country":country}})
        
        
  

        


In [21]:
def checkDups():
    tab = client.prod['techs_test']
    query = tab.find({},{"url":1})
    urls = []
    for i in query: 
        url = i.get("url")
        if url in urls: 
            print("FUCK",url)
            break
        urls.append(url)
    

In [15]:
query = client.prod.model_inputs.find({},{"urls":1,"Country":1,"role":1,"date":1})

missing = 0 
for i in query: 
  
    url = i.get("urls")
    res = client.prod.Scraped_Data.find_one({"url":url})
    
    if res == None: 
        client.prod.Scraped_Data.insert_one({"url":i.get("urls"),"country":i.get("Country"),"date":i.get("date"),"region" : "All"})
        missing  +=1 
print(missing)


0
